In [1]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
df = pd.read_excel(r'Material Data.xlsx')

In [3]:
df.head(20)

,Order Id,Order date,Material,Quantity,UnitPrice,Customer segment
0,1011,14/04/2020,Aluminium,50,14,Small Business
1,1011,14/04/2020,Copper,10,15,Small Business
2,1011,14/04/2020,Zinc,5,16,Small Business
3,1011,14/04/2020,Iron,20,17,Small Business
4,1012,14/05/2020,Copper,10,15,Consumer
5,1012,14/05/2020,Zinc,16,16,Consumer
6,1013,14/05/2020,Copper,25,15,Wholeseller
7,1013,14/05/2020,Zinc,60,16,Wholeseller
8,1013,14/05/2020,Iron,15,17,Wholeseller
9,1014,15/05/2020,Aluminium,20,14,Small Business


In [4]:
df.shape

(200, 6)

In [5]:
df.columns

Index(['Order Id', 'Order date ', 'Material', 'Quantity', 'UnitPrice',
       'Customer segment'],
      dtype='object')

In [28]:
df.groupby('Material').count()

,Order Id,Order date,Quantity,UnitPrice,Customer segment
Material,,,,,
Aluminium,56,56,56,56,56
Copper,52,52,52,52,52
Iron,44,44,44,44,44
Zinc,48,48,48,48,48


In [6]:
df['Material'] = df['Material'].str.strip()
print(df['Material'])
df.dropna(axis=0, subset=['Order Id'], inplace=True)
df['Order Id'] = df['Order Id'].astype('str')
df = df[~df['Order Id'].str.contains('C')]
print(df)

0      Aluminium
1         Copper
2           Zinc
3           Iron
4         Copper
         ...    
195         Zinc
196    Aluminium
197       Copper
198         Zinc
199    Aluminium
Name: Material, Length: 200, dtype: object
    Order Id          Order date    Material  Quantity  UnitPrice  \
0       1011           14/04/2020  Aluminium        50         14   
1       1011           14/04/2020     Copper        10         15   
2       1011           14/04/2020       Zinc         5         16   
3       1011           14/04/2020       Iron        20         17   
4       1012           14/05/2020     Copper        10         15   
..       ...                  ...        ...       ...        ...   
195     1107           29/06/2020       Zinc        26         16   
196     1108           30/06/2020  Aluminium         2         14   
197     1109  2020-01-07 00:00:00     Copper        55         15   
198     1109  2020-01-07 00:00:00       Zinc        56         16   
199     111

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order Id          200 non-null    object
 1   Order date        200 non-null    object
 2   Material          200 non-null    object
 3   Quantity          200 non-null    int64 
 4   UnitPrice         200 non-null    int64 
 5   Customer segment  200 non-null    object
dtypes: int64(2), object(4)
memory usage: 10.9+ KB


In [8]:
df['Customer segment'].unique()

array(['Small Business', 'Consumer', 'Wholeseller'], dtype=object)

In [9]:
group = (df[df['Customer segment'] =="Wholeseller"]
          .groupby(['Order Id', 'Material'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Order Id'))
print(group)

Material  Aluminium  Copper  Iron  Zinc
Order Id                               
1013            0.0    25.0  15.0  60.0
1019            0.0    20.0  15.0   0.0
1022           50.0     0.0   0.0   0.0
1025           50.0    70.0   0.0  80.0
1029           80.0     0.0  20.0  45.0
1033           60.0    55.0  90.0  80.0
1036           55.0    40.0  32.0  45.0
1038           20.0    50.0   0.0   0.0
1040            0.0     0.0   0.0  50.0
1041            0.0     0.0  55.0   0.0
1043            0.0    50.0   0.0   0.0
1045            0.0     0.0   0.0  80.0
1048           60.0    50.0   0.0  55.0
1052            0.0     0.0   0.0  55.0
1054            0.0    50.0   0.0   0.0
1056           60.0     0.0   0.0   0.0
1058           60.0     0.0  55.0   0.0
1062            0.0     0.0  55.0   0.0
1065           55.0     0.0   0.0   0.0
1068           55.0     0.0   0.0   0.0
1070            0.0    55.0  65.0   0.0
1072            0.0    30.0   0.0  55.0
1076           55.0    60.0   0.0  45.0


In [10]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

group_sets = group.applymap(encode_units)
print(group_sets)

Material  Aluminium  Copper  Iron  Zinc
Order Id                               
1013              0       1     1     1
1019              0       1     1     0
1022              1       0     0     0
1025              1       1     0     1
1029              1       0     1     1
1033              1       1     1     1
1036              1       1     1     1
1038              1       1     0     0
1040              0       0     0     1
1041              0       0     1     0
1043              0       1     0     0
1045              0       0     0     1
1048              1       1     0     1
1052              0       0     0     1
1054              0       1     0     0
1056              1       0     0     0
1058              1       0     1     0
1062              0       0     1     0
1065              1       0     0     0
1068              1       0     0     0
1070              0       1     1     0
1072              0       1     0     1
1076              1       1     0     1


In [11]:
frequent_itemsets = apriori(group_sets, min_support=0.08, use_colnames=True)
print(frequent_itemsets)

     support                   itemsets
0   0.580645                (Aluminium)
1   0.483871                   (Copper)
2   0.322581                     (Iron)
3   0.483871                     (Zinc)
4   0.258065        (Aluminium, Copper)
5   0.161290          (Iron, Aluminium)
6   0.258065          (Aluminium, Zinc)
7   0.161290             (Iron, Copper)
8   0.290323             (Copper, Zinc)
9   0.161290               (Iron, Zinc)
10  0.193548  (Aluminium, Copper, Zinc)
11  0.129032    (Iron, Aluminium, Zinc)
12  0.096774       (Iron, Copper, Zinc)


In [12]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Iron),(Copper),0.322581,0.483871,0.161290,0.500000,1.033333,0.005203,1.032258
1,(Copper),(Iron),0.483871,0.322581,0.161290,0.333333,1.033333,0.005203,1.016129
2,(Copper),(Zinc),0.483871,0.483871,0.290323,0.600000,1.240000,0.056191,1.290323
3,(Zinc),(Copper),0.483871,0.483871,0.290323,0.600000,1.240000,0.056191,1.290323
4,(Iron),(Zinc),0.322581,0.483871,0.161290,0.500000,1.033333,0.005203,1.032258
5,(Zinc),(Iron),0.483871,0.322581,0.161290,0.333333,1.033333,0.005203,1.016129
6,"(Aluminium, Copper)",(Zinc),0.258065,0.483871,0.193548,0.750000,1.550000,0.068678,2.064516
7,"(Aluminium, Zinc)",(Copper),0.258065,0.483871,0.193548,0.750000,1.550000,0.068678,2.064516
8,"(Copper, Zinc)",(Aluminium),0.290323,0.580645,0.193548,0.666667,1.148148,0.024974,1.258065
9,(Aluminium),"(Copper, Zinc)",0.580645,0.290323,0.193548,0.333333,1.148148,0.024974,1.064516


In [13]:
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.326) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Iron),(Copper),0.322581,0.483871,0.161290,0.500000,1.033333,0.005203,1.032258
1,(Copper),(Iron),0.483871,0.322581,0.161290,0.333333,1.033333,0.005203,1.016129
2,(Copper),(Zinc),0.483871,0.483871,0.290323,0.600000,1.240000,0.056191,1.290323
3,(Zinc),(Copper),0.483871,0.483871,0.290323,0.600000,1.240000,0.056191,1.290323
4,(Iron),(Zinc),0.322581,0.483871,0.161290,0.500000,1.033333,0.005203,1.032258
5,(Zinc),(Iron),0.483871,0.322581,0.161290,0.333333,1.033333,0.005203,1.016129
6,"(Aluminium, Copper)",(Zinc),0.258065,0.483871,0.193548,0.750000,1.550000,0.068678,2.064516
7,"(Aluminium, Zinc)",(Copper),0.258065,0.483871,0.193548,0.750000,1.550000,0.068678,2.064516
8,"(Copper, Zinc)",(Aluminium),0.290323,0.580645,0.193548,0.666667,1.148148,0.024974,1.258065
9,(Aluminium),"(Copper, Zinc)",0.580645,0.290323,0.193548,0.333333,1.148148,0.024974,1.064516
